In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

input_path = '../data/processed/'

pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Header information

In [5]:
def read_mev_headers(path):
    with open(path, "r") as f:
        mev_header = {}
        for i in range(7):
            line = f.readline().replace("#", "").strip()
            key, value = line.split(":", 1)
            mev_header[key] = value.strip()
    return mev_header

In [6]:
mev_header = read_mev_headers(f"{input_path}test-mev-file.mev")
print(mev_header)

{'Version': 'V1.0', 'Date': 'Tue Apr 22 05:54:18 2008', 'Input_row_count': '4608; bad/undetected spots:1439', 'Created by': 'TIGR Spotfinder 3.1.1, Windows"', 'TIFF files processed': 'ch A- C:/Documents and Settings/Monica Medina/Desktop/Mickey/Mexico2007/19april2008/images/single/bot1 scan4 730 600_532.tif; ch B- C:/Documents and Settings/Monica Medina/Desktop/Mickey/Mexico2007/19april2008/images/single/bot1 scan4 730 600_635.tif;', 'Segmentation method': 'Otsu, Min size: 3, Max size: 100; Top bkg cutoff: 25% "', 'Post-processing settings': 'QCfilter: ON, Background correction: ON, Keep flagged values: ON, QC threshold= 1*medianBKG+1*StdDevBkg"'}


In [7]:
print(mev_header["Version"])

V1.0


# Array data

In [8]:
def read_mev_matix(path, skiprows=8):
    ## read the data from the file
    # skiprows=8 to skip the header ## potential that all mev files have the same header number of rows
    df = pd.read_csv(path, sep="\t", skiprows=skiprows)
    return df

In [9]:
micro_array_data = read_mev_matix(f"{input_path}test-mev-file.mev")
display(micro_array_data.head())

,UID,IA,IB,FeatName,R,C,MR,MC,SR,SC,FlagA,FlagB,SA,SF,QC,QCA,QCB,BkgA,BkgB,SDA,SDB,SDBkgA,SDBkgB,MedA,MedB,MNA,MNB,MedBkgA,MedBkgB,X,Y,PValueA,PValueB
0,1,16742184,16440850,AOSC1103,1,1,1,1,1,1,C,C,428,1,0.95113,0.95113,0.95113,58208,30388,6109.2,6150.6,67.1,54.8,38972,38475,39117,38413,136,71,62,44,0,0
1,2,4113534,3683275,AOSC403,1,2,1,1,1,2,C,C,402,1,0.66650,0.66650,0.66650,51456,26934,2022.2,1708.9,63.4,51.9,10150,9190,10233,9162,128,67,111,44,0,0
2,3,3252615,3046954,AOSC477,1,3,1,1,1,3,C,C,366,1,0.61761,0.61761,0.61761,51240,24888,1952.0,1793.6,68.0,53.1,8637,8255,8887,8325,140,68,160,44,0,0
3,4,4954840,4782054,AOSC568,1,4,1,1,1,4,C,C,382,1,0.71681,0.71681,0.71681,52716,27504,2980.6,2679.4,68.3,54.3,12589,12395,12971,12518,138,72,209,44,0,0
4,5,2290631,1811238,AOSC658,1,5,1,1,1,5,C,C,275,1,0.39633,0.39633,0.39633,39875,18700,2546.1,1943.8,74.1,54.2,7907,6287,8330,6586,145,68,258,44,0,0


In [10]:
print(micro_array_data.columns)

Index(['UID', 'IA', 'IB', 'FeatName', 'R', 'C', 'MR', 'MC', 'SR', 'SC',
       'FlagA', 'FlagB', 'SA', 'SF', 'QC', 'QCA', 'QCB', 'BkgA', 'BkgB', 'SDA',
       'SDB', 'SDBkgA', 'SDBkgB', 'MedA', 'MedB', 'MNA', 'MNB', 'MedBkgA',
       'MedBkgB', 'X', 'Y', 'PValueA', 'PValueB'],
      dtype='object')


In [11]:
print(micro_array_data["FlagA"].value_counts())
print(micro_array_data["FlagB"].value_counts())

FlagA
C    29
Y     1
Name: count, dtype: int64
FlagB
C    29
Y     1
Name: count, dtype: int64


### Flag A and B has multiple potential values for quality control
viewed larger file to check for other potential values

In [12]:
large_mev = read_mev_matix("../data/raw/GSM321647.mev")
print(large_mev["FlagA"].value_counts())
print(large_mev["FlagB"].value_counts())

FlagA
C    3157
X     310
Y      12
Z       7
B       2
S       2
A       1
Name: count, dtype: int64
FlagB
C    3156
X     283
Y      40
Z       7
B       2
S       2
A       1
Name: count, dtype: int64


## Create quality score of Flag values
- 'C' (Compromised): By far the most common flag in your data. This indicates features with some quality concerns, but not necessarily failures. The features might still provide usable data but should be interpreted with caution.
- 'X' (Excluded): These are spots that have been algorithmically determined to be outliers or have failed certain quality metrics. They are typically recommended for exclusion from further analysis.
- 'Y' (Yes/Good quality): Only a small fraction of your spots (12 in channel A, 40 in channel B) were flagged as high-quality without any concerns.
- 'Z' (Zero/Background level): These spots have signal intensities near background levels, suggesting no or very low expression.
- 'B' (Bad): These are spots with severe quality issues, definitely recommended for exclusion.
- 'S' (Saturated): These spots have reached the upper detection limit of the scanner, meaning the true intensity might be higher than what was measured.
- 'A' (Absent): This typically indicates spots where the feature was not detected above background.

In [13]:
def quality_score(row):
    # scoring system to create a 1 for good reads and 0 for reads to be omitted
    # other reads are arbitrary for now
    if row["FlagA"] == "Y" and row["FlagB"] == "Y":
        return 1.0  # Highest quality
    elif row["FlagA"] in ["X", "B", "A"] or row["FlagB"] in ["X", "B", "A"]:
        return 0.0  # Poor quality, consider excluding
    elif row["FlagA"] == "C" or row["FlagB"] == "C":
        return 0.5  # Compromised but potentially usable
    elif row["FlagA"] == "Z" or row["FlagB"] == "Z":
        return 0.3  # Near background, low confidence
    else:
        return 0.8  # Other flags


# df['quality_score'] = df.apply(quality_score, axis=1)

In [14]:
# IB/IA
print(np.log2(1033136 / 1145362))

-0.14877346441283357


In [15]:
def mev_to_expression_matrix(mev_data):
    mev_data_copy = mev_data.copy()
    mev_data_copy["log2_expression"] = np.log2(
        mev_data_copy["IB"] / mev_data_copy["IA"]
    )
    mev_data_copy = mev_data_copy[["FeatName", "log2_expression"]]
    return mev_data_copy

In [16]:
expression_matrix = mev_to_expression_matrix(micro_array_data)
display(expression_matrix.head())

,FeatName,log2_expression
0,AOSC1103,-0.026203
1,AOSC403,-0.159389
2,AOSC477,-0.094232
3,AOSC568,-0.051208
4,AOSC658,-0.338769


In [17]:
from mev_to_csv import MevToCsv

In [18]:
mev_file = "../data/raw/GSM321647.mev"
output_path = "../data/processed/GSM321647.csv"

In [19]:
converter = MevToCsv(mev_file, quality_score=True)
mev_header, matrix = converter.convert()

Excluded 314 samples due to low quality scores.


In [20]:
matrix

,FeatName,GSM321647
0,AOSC1103,0.506580
1,AOSC403,0.540947
2,AOSC477,0.523943
3,AOSC568,0.512916
4,AOSC658,0.589653
...,...,...
3476,CAXB2116,0.543708
3477,CAXB2318,0.577526
3478,CAXB2491,0.491691
3480,Apal mitochondrial gene,0.581827


## Test expression matrix builder

In [21]:
from mev_to_csv import ExpressionMatrixBuilder

In [26]:
mev_paths = ExpressionMatrixBuilder.paths_builder("../data/raw/")
mev_paths

['../data/raw/GSM321647.mev',
 '../data/raw/GSM321673.mev',
 '../data/raw/GSM321677.mev']

In [35]:
builder = ExpressionMatrixBuilder(mev_paths, quality_score=True)
headers, matrices = builder.read_mevs()
merged_matrix = builder.merge_mevs()
builder.save_merged_matrix("../data/processed/merged_matrix.csv")
merged_matrix

Excluded 314 samples due to low quality scores.
Excluded 496 samples due to low quality scores.
Excluded 511 samples due to low quality scores.
⚠️ Warning: Duplicate sample identifiers found: FeatName, FeatName
⚠️ 615 missing values were filled with 0.
⚠️ Warning: Duplicate sample identifiers found: FeatName, FeatName
⚠️ 615 missing values were filled with 0.


,FeatName,GSM321647,GSM321673,GSM321677
0,AOSC1103,0.506580,0.616808,0.468331
1,AOSC403,0.540947,0.520399,0.409619
2,AOSC477,0.523943,0.506037,0.352103
3,AOSC568,0.512916,0.471960,0.299529
4,AOSC658,0.589653,0.519331,0.502956
...,...,...,...,...
3360,0,0.000000,0.512093,0.000000
3489,0,0.000000,0.000000,0.000000
3490,0,0.000000,0.000000,0.000000
2574,0,0.000000,0.000000,0.000000


In [40]:
from mev_to_csv import process_mev_folder

# Simple one-line usage
result = process_mev_folder("../data/raw/", "../data/processed/merged_matrix.csv")
result

Excluded 314 samples due to low quality scores.
Excluded 496 samples due to low quality scores.
Excluded 511 samples due to low quality scores.
⚠️ Warning: Duplicate sample identifiers found: FeatName, FeatName
⚠️ 615 missing values were filled with 0.
⚠️ Warning: Duplicate sample identifiers found: FeatName, FeatName
⚠️ 615 missing values were filled with 0.


,FeatName,GSM321647,GSM321673,GSM321677
0,AOSC1103,0.506580,0.616808,0.468331
1,AOSC403,0.540947,0.520399,0.409619
2,AOSC477,0.523943,0.506037,0.352103
3,AOSC568,0.512916,0.471960,0.299529
4,AOSC658,0.589653,0.519331,0.502956
...,...,...,...,...
3360,0,0.000000,0.512093,0.000000
3489,0,0.000000,0.000000,0.000000
3490,0,0.000000,0.000000,0.000000
2574,0,0.000000,0.000000,0.000000
